# Cargar librerias

In [20]:
import pandas as pd
import numpy as np
import openpyxl
import os
from openpyxl import Workbook
from openpyxl.utils.dataframe import dataframe_to_rows
from datetime import datetime, timedelta
import warnings

# Cargar bases de datos 

In [21]:
Plano532_COGN = "D:/OneDrive - Allianz/6_Procesos TRANSMISIONES/FORMATO_532/Bases/Plano532_COGN.xlsx"
df = pd.read_excel(Plano532_COGN, header=10, skiprows=[11])
df = df.dropna(axis=1, how='all')
df = df[:-2]

In [22]:
ruta_excel = "D:/OneDrive - Allianz/6_Procesos TRANSMISIONES/Fuentes de Información COGN.xlsx"
RESERVAS_VP = pd.read_excel(ruta_excel, sheet_name="RESERVAS_VP")

In [23]:
ruta_excel = "D:/OneDrive - Allianz/6_Procesos TRANSMISIONES/Fuentes de Información COGN.xlsx"

BALANCE = pd.read_excel(ruta_excel,sheet_name="BALANCE_WP",usecols="A:E",skiprows=5,dtype=str)
BALANCE.columns = ["CUENTA", "NOMBRE", "VALOR", "VALOR2", "VALOR MONEDA EXTRANJERA"]

# VALIDACIONES 

In [28]:
# Cargar los datos necesarios del archivo Plano532_COGN
Plano532_df = pd.read_excel(Plano532_COGN)
Plano_ramos = Plano532_df.iloc[9, 3:33]  # Fila 9, Columnas D a AG para los nombres de Ramos
Plano_values = Plano532_df.iloc[14, 3:33]  # Fila 14, Columnas D a AG para los valores de reserva

# Crear un DataFrame para los datos de Plano532_COGN
Plano_df = pd.DataFrame({
    'Ramo': Plano_ramos.values, 
    'Valor reserva técnica de prima no devengada': Plano_values.values
})

# Normalizar los nombres de ramos en Plano_df
Plano_df['Ramo'] = Plano_df['Ramo'].astype(str).str.strip().str.upper()

# Cargar los datos necesarios de la hoja "resumen" en RESERVAS_VP
RESERVAS_ramos = RESERVAS_VP.iloc[45:66, 1].values  # Columna B
RESERVAS_values = abs(RESERVAS_VP.iloc[45:66, 2].values)  # Columna C
insuficiencia_primas = abs(RESERVAS_VP.iloc[45:66, 5].values)  # Columna F

# Crear un DataFrame para los datos de RESERVAS_VP incluyendo insuficiencia de primas
RESERVAS_df = pd.DataFrame({
    'Ramo': RESERVAS_ramos, 
    'Resumen Valor reserva técnica de prima no devengada': RESERVAS_values,
    'Insuficiencia de primas': insuficiencia_primas
})

# Normalizar los nombres de ramos en RESERVAS_df
RESERVAS_df['Ramo'] = RESERVAS_df['Ramo'].astype(str).str.strip().str.upper()

# Unir ambos DataFrames en base a la columna "Ramo", manteniendo todos los valores de Plano_df
Validacion_df = pd.merge(Plano_df, RESERVAS_df, on="Ramo", how="left")

# Asegurarse de que las columnas tengan el tipo de datos correcto antes de usar fillna
Validacion_df['Valor reserva técnica de prima no devengada'] = Validacion_df['Valor reserva técnica de prima no devengada'].astype(float).fillna(0)
Validacion_df['Resumen Valor reserva técnica de prima no devengada'] = Validacion_df['Resumen Valor reserva técnica de prima no devengada'].astype(float).fillna(0)
Validacion_df['Insuficiencia de primas'] = Validacion_df['Insuficiencia de primas'].astype(float).fillna(0)

# Calcular la diferencia entre ambos valores de reserva
Validacion_df['Diferencia'] = Validacion_df['Valor reserva técnica de prima no devengada'] - Validacion_df['Resumen Valor reserva técnica de prima no devengada']

# Calcular los totales de cada columna numérica
total_reserva = Validacion_df['Valor reserva técnica de prima no devengada'].sum()
total_resumen = Validacion_df['Resumen Valor reserva técnica de prima no devengada'].sum()
total_insuficiencia = Validacion_df['Insuficiencia de primas'].sum()
total_diferencia = Validacion_df['Diferencia'].sum()

# Crear una fila de totales y agregarla al DataFrame
totales = pd.DataFrame({
    'Ramo': ['TOTAL'],
    'Valor reserva técnica de prima no devengada': [total_reserva],
    'Resumen Valor reserva técnica de prima no devengada': [total_resumen],
    'Insuficiencia de primas': [total_insuficiencia],
    'Diferencia': [total_diferencia]
})
Validacion_df = pd.concat([Validacion_df, totales], ignore_index=True)

# Agregar una fila en blanco entre TOTAL y CUENTA, omitiendo columnas con todos los valores None
fila_en_blanco = pd.DataFrame({
    'Ramo': [''],
    'Valor reserva técnica de prima no devengada': [None],
    'Resumen Valor reserva técnica de prima no devengada': [None],
    'Insuficiencia de primas': [None],
    'Diferencia': [None]
}).dropna(how="all", axis=1)
Validacion_df = pd.concat([Validacion_df, fila_en_blanco], ignore_index=True)

# Convertir la columna "VALOR" a string y luego a numérico para hacer cálculos
BALANCE["VALOR"] = pd.to_numeric(BALANCE["VALOR"].astype(str).str.replace(",", ""), errors="coerce")

# Extraer el valor de la cuenta "261000"
cuenta_valor = BALANCE[BALANCE["CUENTA"] == "261000"]["VALOR"].values[0]

# Crear filas para mostrar el número de la cuenta, "Valor de Cuenta" y "Valor de resumen" en la columna B
cuenta_filas = pd.DataFrame({
    'Ramo': ['CUENTA', 'Valor de Cuenta', 'Valor de resumen'],
    'Valor reserva técnica de prima no devengada': [261000, cuenta_valor, None],
    'Resumen Valor reserva técnica de prima no devengada': [None, None, None],  # Mantener la columna vacía
    'Insuficiencia de primas': [None, None, None],
    'Diferencia': [None, None, None]
}).dropna(how="all", axis=1)

# Agregar el total de "Valor de resumen" en la columna B en la última fila
cuenta_filas.loc[2, 'Valor reserva técnica de prima no devengada'] = total_resumen + total_insuficiencia

# Concatenar las filas de cuenta al DataFrame de Validación
Validacion_df = pd.concat([Validacion_df, cuenta_filas], ignore_index=True)

# Guardar el archivo Excel final con la nueva hoja "VALIDACIONES" dentro del archivo Plano532_COGN
with pd.ExcelWriter(Plano532_COGN, mode="a", engine="openpyxl", if_sheet_exists="replace") as writer:
    Validacion_df.to_excel(writer, index=False, sheet_name="VALIDACIONES")


## PROFORMA TXT

In [29]:
# Definir las columnas clave
columna_subcuenta = 'SUBCUENTA'
columna_unidad_captura = 'UNIDAD DE CAPTURA'
columna_concepto = 'CONCEPTO'  # Añadir aquí la columna que contiene texto para omitirla

# Inicializar la lista para almacenar las líneas del archivo TXT
txt_datos = []

# Obtener la fecha correcta (mes vencido)
fecha_actual = datetime.now()
fecha_mes_vencido = fecha_actual.replace(day=1) - timedelta(days=1)
fecha_str = fecha_mes_vencido.strftime("%d%m%Y")  # Formato día-mes-año

# Encabezados fijos de la primera y segunda línea
registro_inicial = f"00001113000001{fecha_str}00118SGENCOLASEG0184"
segunda_linea = "000022000001100000"

# Agregar las dos primeras líneas a la lista `txt_datos`
txt_datos.append(registro_inicial)
txt_datos.append(segunda_linea)

# Filtrar las filas que no tengan datos válidos en la columna 'SUBCUENTA'
df = df.dropna(subset=[columna_subcuenta])

# Función para formatear la línea de la cuarta parte
def formatear_linea_cuarta_parte(numero_registro, codigo_formato, columna_num, unidad_captura, subcuenta, signo, valor):
    num_registro_str = str(numero_registro).zfill(5)  # Asegurar 5 dígitos para el número de registro
    tipo_registro = "4"  # Tipo de registro siempre debe ser 1 dígito
    codigo_formato_str = str(codigo_formato).zfill(3)  # 3 dígitos para el código del formato
    columna_str = f"{columna_num:02d}"  # Asegurar que el número de columna sea un entero con 2 dígitos
    unidad_captura_str = f"{int(float(unidad_captura)):02d}"  # Convertir a entero y asegurar 2 dígitos para la unidad de captura
    subcuenta_str = f"{int(float(subcuenta)):03d}"  # Convertir a entero y asegurar 3 dígitos para la subcuenta
    valor_str = f"{int(valor):020d}"  # Valor formateado a 20 caracteres con ceros a la izquierda
    return f"{num_registro_str}{tipo_registro}{codigo_formato_str}{columna_str}{unidad_captura_str}{subcuenta_str}{signo}{valor_str}"

# Inicializar el número de registro
numero_registro = 3  # Iniciar en 3 porque las dos primeras líneas ya están fijas

# Recorrer las columnas y filas para generar las líneas, asegurando que el número de columna se incremente correctamente
for columna_num, columna in enumerate(df.columns, start=-1):  # 'start=-1' para iniciar en 01
    # Omitir las columnas 'SUBCUENTA', 'UNIDAD DE CAPTURA' y 'CONCEPTO'
    if columna not in [columna_subcuenta, columna_unidad_captura, columna_concepto]:
        for _, row in df.iterrows():
            subcuenta = row[columna_subcuenta]
            unidad_captura = row[columna_unidad_captura]
            valor = row[columna]

            # Saltar si el valor es cero o NaN
            if pd.isna(valor) or valor == 0:
                continue

            # Determinar el signo del valor
            try:
                valor_float = float(valor)
                signo = '+' if valor_float >= 0 else '-'
                valor_abs = abs(valor_float)

                # Formatear la línea específica para el Formato 532
                linea = formatear_linea_cuarta_parte(numero_registro, 532, columna_num, unidad_captura, subcuenta, signo, valor_abs)
                txt_datos.append(linea)

                numero_registro += 1

            except ValueError:
                # Omitir el error y continuar sin imprimir mensajes para valores no numéricos
                pass

# Agregar la última línea con el conteo de registros y un "5" al final
total_registros = len(txt_datos) + 1  # Contar todas las líneas generadas
linea_final = f"{str(total_registros).zfill(5)}5"  # Formatear el total con un "5" al final
txt_datos.append(linea_final)

# Guardar el archivo TXT final con todas las partes sin agregar un salto de línea al final
with open("Plano532_GEN.txt", "w") as f:
    # Escribir todas las líneas excepto la última con un salto de línea
    for linea in txt_datos[:-1]:
        f.write(f"{linea}\n")
    # Escribir la última línea sin un salto de línea adicional
    f.write(txt_datos[-1])
